# Data Audit

This script takes an audit of the EPC data for Wales. It looks at how many missing values each feature has, how many unique values the categorical features have then applies preprocessing steps to remove outliers and drop features with too many missing values.

In [1]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt

In [2]:
# set variables from config file
config_path = os.path.abspath('..')[:-7]

with open(config_path + '/config.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
epc_wales_fname = config['DEFAULT']['epc_wales_fname']
epc_train_fname = config['DEFAULT']['epc_train_fname']
epc_test_fname = config['DEFAULT']['epc_test_fname']

In [3]:
#Specify ther import dictionary for the dates
dtype_dict = {'INSPECTION_DATE':'str'}

epc_train = pd.read_csv(os.path.join(processing_path,epc_train_fname), header = 0, delimiter = ',', 
                       dtype = dtype_dict, parse_dates = ['INSPECTION_DATE'])

epc_test = pd.read_csv(os.path.join(processing_path,epc_test_fname),header = 0, delimiter = ',',
                        dtype = dtype_dict, parse_dates = ['INSPECTION_DATE'])

In [22]:
epc_train['inspection_year'] = epc_train['INSPECTION_DATE'].dt.year

In [24]:
epc_train['inspection_year'].value_counts(normalize=True)

2013    0.142041
2014    0.141789
2009    0.133969
2015    0.112991
2011    0.108753
2010    0.099550
2012    0.096038
2008    0.093646
2016    0.070897
2007    0.000272
2006    0.000044
2001    0.000003
2005    0.000002
2004    0.000002
2003    0.000002
2000    0.000002
Name: inspection_year, dtype: float64

In [4]:
print(len(epc_train))
print(len(epc_test))

632607
158152


### Investigate categoric and numeric fields

In [5]:
# Get numeric and categorical variable names 
var_list_num = epc_train.select_dtypes(include= 'number').columns.tolist()

var_list_cat = epc_train.select_dtypes(include= ['object','category']).columns.tolist()
var_list_cat.remove('LMK_KEY')
var_list_cat.remove('POSTCODE')

In [6]:
# Percentage of missing values in numeric fields
epc_train[var_list_num].isnull().sum().sort_values(ascending = False) / epc_train.shape[0]

COUNTY                           1.000000
FLAT_STOREY_COUNT                0.920668
UNHEATED_CORRIDOR_LENGTH         0.916084
FLOOR_HEIGHT                     0.458915
MULTI_GLAZE_PROPORTION           0.127310
NUMBER_HEATED_ROOMS              0.088380
EXTENSION_COUNT                  0.088380
NUMBER_HABITABLE_ROOMS           0.086953
NUMBER_OPEN_FIREPLACES           0.085348
LOW_ENERGY_LIGHTING              0.063464
CO2_EMISSIONS_CURRENT            0.000000
CURRENT_ENERGY_EFFICIENCY        0.000000
ENERGY_CONSUMPTION_CURRENT       0.000000
TOTAL_FLOOR_AREA                 0.000000
CO2_EMISS_CURR_PER_FLOOR_AREA    0.000000
LIGHTING_COST_CURRENT            0.000000
HEATING_COST_CURRENT             0.000000
HOT_WATER_COST_CURRENT           0.000000
BUILDING_REFERENCE_NUMBER        0.000000
dtype: float64

Drop COUNTY, FLAT_STOREY_COUNT AND UNHEATED_CORRIDOR_LENGTH due to high percentage of 
missing values. Keep FLOOR_HEIGHT but be careful

In [7]:
audit_num = epc_train[var_list_num].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

We have some negative values and some massively large

Clip all below 0

Clip all at the 95th percentile with the exception of the following

In [8]:
# Percentage of missing values in categoric fields
epc_train[var_list_cat].isnull().sum().sort_values(ascending = False) / epc_train.shape[0]

SHEATING_ENERGY_EFF         0.987942
SHEATING_ENV_EFF            0.987942
FLOOR_ENERGY_EFF            0.943562
FLOOR_ENV_EFF               0.943562
HEAT_LOSS_CORRIDOOR         0.834324
FLAT_TOP_STOREY             0.834324
FLOOR_LEVEL                 0.816509
SOLAR_WATER_HEATING_FLAG    0.511483
GLAZED_TYPE                 0.126036
ROOF_ENERGY_EFF             0.108116
ROOF_ENV_EFF                0.108116
MAINS_GAS_FLAG              0.089397
MECHANICAL_VENTILATION      0.088380
ENERGY_TARIFF               0.072122
TRANSACTION_TYPE            0.051751
BUILT_FORM                  0.035211
MAIN_FUEL                   0.021266
SECONDHEAT_DESCRIPTION      0.015074
HOT_WATER_ENV_EFF           0.014831
MAINHEATCONT_DESCRIPTION    0.014696
MAIN_HEATING_CONTROLS       0.014696
HOTWATER_DESCRIPTION        0.014535
HOT_WATER_ENERGY_EFF        0.014186
ROOF_DESCRIPTION            0.013310
WALLS_DESCRIPTION           0.013264
FLOOR_DESCRIPTION           0.013221
WINDOWS_DESCRIPTION         0.013182
L

Drop SHEATING_ENERGY_EFF, SHEATING_ENV_EFF, FLOOR_ENV_EFF, FLAT_TOP_STOREY, FLOOR_ENERGY_EFF
and SOLAR_WATER_HEATING_FLAG due to percentage of missing values

In [9]:
audit_cat = epc_train[var_list_cat].describe()
epc_train[var_list_cat].nunique().sort_values(ascending = False)

LODGEMENT_DATE              3189
WALLS_DESCRIPTION            837
ROOF_DESCRIPTION             720
FLOOR_DESCRIPTION            427
LIGHTING_DESCRIPTION         278
SECONDHEAT_DESCRIPTION       150
HOTWATER_DESCRIPTION         125
MAIN_HEATING_CONTROLS         89
MAINHEATCONT_DESCRIPTION      89
WINDOWS_DESCRIPTION           48
MAIN_FUEL                     37
FLOOR_LEVEL                   26
region                        22
TRANSACTION_TYPE              14
GLAZED_TYPE                    9
ENERGY_TARIFF                  8
CURRENT_ENERGY_RATING          8
BUILT_FORM                     6
HOT_WATER_ENERGY_EFF           5
HOT_WATER_ENV_EFF              5
PROPERTY_TYPE                  5
FLOOR_ENERGY_EFF               5
FLOOR_ENV_EFF                  5
WINDOWS_ENV_EFF                5
WINDOWS_ENERGY_EFF             5
ROOF_ENV_EFF                   5
LIGHTING_ENV_EFF               5
LIGHTING_ENERGY_EFF            5
MAINHEATC_ENV_EFF              5
MAINHEATC_ENERGY_EFF           5
MAINHEAT_E

We need to reduce the levels

### Drop unwanted fields

In [10]:
epc_train.drop(['COUNTY','FLAT_STOREY_COUNT','UNHEATED_CORRIDOR_LENGTH','LODGEMENT_DATE',
                'SHEATING_ENERGY_EFF','SHEATING_ENV_EFF','FLOOR_ENV_EFF',
                'FLAT_TOP_STOREY','FLOOR_ENERGY_EFF','SOLAR_WATER_HEATING_FLAG'],
                axis = 1,
                inplace = True)

epc_test.drop(['COUNTY','FLAT_STOREY_COUNT','UNHEATED_CORRIDOR_LENGTH','LODGEMENT_DATE',
               'SHEATING_ENERGY_EFF','SHEATING_ENV_EFF','FLOOR_ENV_EFF',
               'FLAT_TOP_STOREY','FLOOR_ENERGY_EFF','SOLAR_WATER_HEATING_FLAG'],
               axis = 1,
               inplace = True)

### Clip training and test datasets

In [15]:
def call_clip(df):
    
    ''' 
    Clips the variables of the EPC data between 0 and a specified upper amount
    Parameters
      df: a dataframe containing EPC data specified below
    Returns the dataframe with clipped values
    '''
    
    df['CURRENT_ENERGY_EFFICIENCY'].clip(lower = 0.0, upper = 100, inplace = True)
    df['ENERGY_CONSUMPTION_CURRENT'].clip(lower = 0.0, upper = 600, inplace = True)
    df['CO2_EMISSIONS_CURRENT'].clip(lower = 0.0, upper = 20, inplace = True)
    df['CO2_EMISS_CURR_PER_FLOOR_AREA'].clip(lower = 0.0, upper = 100, inplace = True)
    df['LIGHTING_COST_CURRENT'].clip(lower = 0.0, upper = 150, inplace = True)
    df['HEATING_COST_CURRENT'].clip(lower = 0.0, upper = 1750, inplace = True)
    df['HOT_WATER_COST_CURRENT'].clip(lower = 0.0, upper = 350, inplace = True)
    df['TOTAL_FLOOR_AREA'].clip(lower = 0.0, upper = 175, inplace = True)
    df['MULTI_GLAZE_PROPORTION'].clip(lower = 0.0, upper = 100, inplace = True)
    df['EXTENSION_COUNT'].clip(lower = 0.0, upper = 4, inplace = True)
    df['NUMBER_HABITABLE_ROOMS'].clip(lower = 0.0, upper = 10, inplace = True)
    df['NUMBER_HEATED_ROOMS'].clip(lower = 0.0, upper = 9, inplace = True)
    df['LOW_ENERGY_LIGHTING'].clip(lower = 0.0, upper = 100, inplace = True)
    df['NUMBER_OPEN_FIREPLACES'].clip(lower = 0.0, upper = 4, inplace = True)
    df['FLOOR_HEIGHT'].clip(lower = 0.0, upper = 3, inplace = True)
    return(df)

epc_train = call_clip(epc_train)
epc_test = call_clip(epc_test)

### Export datasets

In [16]:
epc_train.to_csv(os.path.join(processing_path,epc_train_fname),index = False)
epc_test.to_csv(os.path.join(processing_path,epc_test_fname),index = False)